# NetID:  gg676, xl598, vt152, smk371

In [ ]:
from PIL import Image
import os
import re
import time
import sys
import glob
import pickle
from pathlib import Path
import numpy as np
from skimage import io

from torch.utils.data import DataLoader,Dataset
#import lmdb
import torch
import torch.nn as nn


In [ ]:


root_path = 'C:/govind/536/test/images/'

transform = transforms.Compose([transforms.Resize(256),
                                transforms.CenterCrop(224),
                               transforms.ToTensor(),
                               transforms.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225])])


def list_all_images(path):
    img_list = []
    id_list = []
    for i_1 in glob.glob(path+'/*'):
        for i_2 in glob.glob(i_1+'/*'):
            for i_3 in glob.glob(i_2+'/*'):
                for i_4 in glob.glob(i_3+'/*'):
                    img_list.append(i_4)
                    #m = re.search()
                    id_list.append(Path(i_4).stem)
                    #im = Image.open(i_4)
                    #img_list.append(im)
                    #im.close()
    return id_list, img_list

def list_all_image_directories(root_path):
    return glob.glob(root_path+'*')



class ImageDataset(Dataset):
    def __init__(self, id_list, img_list, transform):
        self.transform = transform
        self.id_list = id_list
        self.img_list = img_list

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, index):
        img = Image.open(self.img_list[index])

        image = self.transform(img)

        return self.id_list[index], image


class new_model(nn.Module):
    def __init__(self, output_layer):
        super().__init__()
        self.output_layer = output_layer
        self.pretrained = models.resnet50(pretrained=True)
        self.children_list = []
        for n, c in self.pretrained.named_children():
            self.children_list.append(c)
            if n == self.output_layer:
                break

        self.net = nn.Sequential(*self.children_list)
        self.pretrained = None

    def forward(self, x):
        with torch.no_grad():
            x = self.net(x)
        return x

def extract_features(model, dataloader):
    start = time.time()
    feature_tensor_array = torch.tensor([])
    id_list = []
    for idx, batch_data in enumerate(dataloader):
        #print(batch_data.shape)
        batch_id, batch_data = batch_data
        batch_data = batch_data.to('cuda')
        output = model(batch_data)
        output = output.squeeze(-1)
        output = output.squeeze(-1)
        #print(output.shape)
        id_list.extend(batch_id)
        feature_tensor_array = torch.cat([feature_tensor_array, output.detach().cpu()], dim=0)
    print("Time taken: ", time.time() - start)
    return id_list, feature_tensor_array

def save_features(path, file_name, data):
    torch.save(data, path+file_name)
    #with open(path+file_name, 'wb') as fp:
    #    pickle.dump(data, fp)


def main():
    all_image_directories = list_all_image_directories(root_path)
    model = new_model(output_layer='avgpool')
    model = model.to('cuda')
    # print(all_image_directories)
    for i, directory_path in enumerate(all_image_directories):
        #feature_list = []
        #ids_list = []
        print("Count: ", i)

        id_list, img_list = list_all_images(directory_path)
        dataset = ImageDataset(id_list, img_list, transform)
        dataloader = DataLoader(dataset, batch_size=384, shuffle=False, num_workers=5)
        ids_list, feature_list = extract_features(model, dataloader)
        del dataset
        del dataloader
        save_features('C:/govind/536/test/persisted/', 'id_list_' + str(i)+'.pth', ids_list)
        save_features('C:/govind/536/test/persisted/', 'feature_list_' + str(i)+'.pth', feature_list)
    # return feature_list

if __name__ == '__main__':
    main()
